In [8]:
import pandas as pd
import numpy as np
from sqlalchemy  import create_engine
from faker import Faker
from datetime import datetime,timedelta,date
from dateutil.relativedelta import relativedelta

In [9]:
fake = Faker('zh_CN')
def generate_fixed_asset_data(num_assets=500):
    # 资产主台账
    assets = []
    for i in range(num_assets):
        purchase_date = fake.date_between(start_date='-10y', end_date='-1y')
        useful_life = np.random.choice([5, 10, 15, 20])
        status = '在用' if (date.today() - purchase_date).days / 365.25 < useful_life else '报废'
        if np.random.rand() < 0.1: status = '闲置'

        assets.append({
            'asset_id': f'FA-{10000+i}',
            'asset_name': f'{np.random.choice(["精密机床", "服务器", "运输卡车", "办公电脑"])} #{i}',
            'asset_category': np.random.choice(["生产设备", "电子设备", "运输工具"]),
            'purchase_cost': round(np.random.lognormal(11, 1.5), 2),
            'purchase_date': purchase_date,
            'useful_life_years': useful_life,
            'salvage_rate': 0.05,
            'asset_status': status,
            'location': fake.province(),
            'department': fake.company_suffix() + "部"
        })
    asset_df = pd.DataFrame(assets)

    # 折旧、维保、处置记录
    dep_records, maint_records, disp_records = [], [], []
    for _, asset in asset_df.iterrows():
        depreciable_base = asset['purchase_cost'] * (1 - asset['salvage_rate'])
        monthly_dep = depreciable_base / (asset['useful_life_years'] * 12)
        
        # 植入异常：折旧计算错误
        if np.random.rand() < 0.05:
            monthly_dep *= 1.2 # 错误地多计提20%
        
        acc_dep = 0
        current_date = asset['purchase_date']
        end_date = date.today()

        while current_date < end_date:
            period_str = current_date.strftime('%Y-%m')
            
            # 折旧
            if acc_dep < depreciable_base:
                acc_dep += monthly_dep
                dep_records.append({
                    'asset_id': asset['asset_id'],
                    'period': period_str,
                    'monthly_depreciation': round(monthly_dep, 2),
                    'accumulated_depreciation': round(acc_dep, 2)
                })

            # 维保
            # 植入异常：幽灵资产
            if np.random.rand() < (0.01 if asset['asset_status'] == '在用' else 0.001):
                 maint_records.append({
                    'asset_id': asset['asset_id'],
                    'maintenance_date': current_date,
                    'maintenance_cost': round(asset['purchase_cost'] * 0.01 * np.random.rand(), 2),
                    'details': '常规检查'
                })

            current_date += relativedelta(months=1)
        
        # 处置
        if asset['asset_status'] == '报废':
            nbv = asset['purchase_cost'] - acc_dep
            proceeds = asset['purchase_cost'] * asset['salvage_rate'] * (1 + (np.random.rand() - 0.5) * 0.2)
            disp_records.append({
                'asset_id': asset['asset_id'],
                'disposal_date': asset['purchase_date'] + relativedelta(years=asset['useful_life_years']),
                'disposal_method': '报废',
                'proceeds': round(proceeds,2),
                'net_book_value_at_disposal': round(nbv, 2),
                'net_gain_loss': round(proceeds - nbv, 2)
            })
            
    return asset_df, pd.DataFrame(dep_records), pd.DataFrame(maint_records), pd.DataFrame(disp_records)

assets, depreciation, maintenance, disposals = generate_fixed_asset_data()
print("固定资产模拟数据已生成。")



固定资产模拟数据已生成。


In [11]:
db_user='root'
db_password='123456'
db_host='localhost'
db_port='3306'
db_name='fixed_asset_db'
try:
    engine=create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    assets.to_sql('asset_register',engine,if_exists='append',index=False)
    depreciation.to_sql('depreciation_schedule',engine,if_exists='append',index=False)
    maintenance.to_sql('maintenance_logs',engine,if_exists='append',index=False)
    disposals.to_sql('disposal_records',engine,if_exists='append',index=False)
    print('数据已成功写入数据库')
except Exception as e:
    print(f'发生错误:{e}')

数据已成功写入数据库
